In [1]:
import datetime
# import plotly packages 
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use("seaborn-whitegrid")
%matplotlib inline
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.model_selection import train_test_split, KFold
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler,LabelEncoder
le=LabelEncoder()
#pd.options.display.max_rows=None
plt.style.use("seaborn-whitegrid")
import warnings
warnings.filterwarnings("ignore")
from tqdm.notebook import tqdm_notebook
ls=pd.read_excel('Lemonade_sales_data_SET.xlsx')
ls

C:\Users\agboola_yusuf\AppData\Local\Temp\ipykernel_936\865340377.py:9: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use("seaborn-whitegrid")
C:\Users\agboola_yusuf\AppData\Local\Temp\ipykernel_936\865340377.py:19: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use("seaborn-whitegrid")


,Date,Day of the Week,Temperature (°F),Special Event,Sales ($)
0,2023-06-01,Thursday,80,0,55.33
1,2023-06-02,Friday,74,0,47.17
2,2023-06-03,Saturday,81,0,50.24
3,2023-06-04,Sunday,90,0,61.28
4,2023-06-05,Monday,73,0,39.73
...,...,...,...,...,...
85,2023-08-25,Friday,70,0,43.43
86,2023-08-26,Saturday,84,0,59.32
87,2023-08-27,Sunday,78,0,59.93
88,2023-08-28,Monday,70,0,38.00


# Creating a new column "Day Number" to numerically represent each day sequentially.


In [2]:
ls['Day Number'] = range(1, len(ls) + 1)
ls

,Date,Day of the Week,Temperature (°F),Special Event,Sales ($),Day Number
0,2023-06-01,Thursday,80,0,55.33,1
1,2023-06-02,Friday,74,0,47.17,2
2,2023-06-03,Saturday,81,0,50.24,3
3,2023-06-04,Sunday,90,0,61.28,4
4,2023-06-05,Monday,73,0,39.73,5
...,...,...,...,...,...,...
85,2023-08-25,Friday,70,0,43.43,86
86,2023-08-26,Saturday,84,0,59.32,87
87,2023-08-27,Sunday,78,0,59.93,88
88,2023-08-28,Monday,70,0,38.00,89


# preparing data  for forecasting

In [3]:
#label encoding for day of week
ls['Day of the Week'] = le.fit_transform(ls['Day of the Week'])
# conversion to acceptable format and extracting :day,year,month,day,and so on
ls['Date']=pd.to_datetime(ls['Date'])
ls['year'] = pd.to_datetime(ls['Date']).dt.year
ls['month'] = pd.to_datetime(ls['Day of the Week']).dt.month
ls['weekofyear'] = pd.to_datetime(ls['Day of the Week']).dt.weekofyear
ls['day'] = pd.to_datetime(ls['Day of the Week']).dt.day
# calculating of sine and cosine for time series to capture trend and enhance prediction
def encode_date(df, col, max_val):
    df[col + '_sin'] = np.sin(2 * np.pi * df[col]/max_val)
    df[col + '_cos'] = np.cos(2 * np.pi * df[col]/max_val)
    return df

encode_date(ls, 'weekofyear', 53)
encode_date(ls, 'month', 13)
encode_date(ls, 'day', 32)

,Date,Day of the Week,Temperature (°F),Special Event,Sales ($),Day Number,year,month,weekofyear,day,weekofyear_sin,weekofyear_cos,month_sin,month_cos,day_sin,day_cos
0,2023-06-01,4,80,0,55.33,1,2023,1,1,1,0.118273,0.992981,0.464723,0.885456,0.19509,0.980785
1,2023-06-02,0,74,0,47.17,2,2023,1,1,1,0.118273,0.992981,0.464723,0.885456,0.19509,0.980785
2,2023-06-03,2,81,0,50.24,3,2023,1,1,1,0.118273,0.992981,0.464723,0.885456,0.19509,0.980785
3,2023-06-04,3,90,0,61.28,4,2023,1,1,1,0.118273,0.992981,0.464723,0.885456,0.19509,0.980785
4,2023-06-05,1,73,0,39.73,5,2023,1,1,1,0.118273,0.992981,0.464723,0.885456,0.19509,0.980785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,2023-08-25,0,70,0,43.43,86,2023,1,1,1,0.118273,0.992981,0.464723,0.885456,0.19509,0.980785
86,2023-08-26,2,84,0,59.32,87,2023,1,1,1,0.118273,0.992981,0.464723,0.885456,0.19509,0.980785
87,2023-08-27,3,78,0,59.93,88,2023,1,1,1,0.118273,0.992981,0.464723,0.885456,0.19509,0.980785
88,2023-08-28,1,70,0,38.00,89,2023,1,1,1,0.118273,0.992981,0.464723,0.885456,0.19509,0.980785


# every column is in numerial data types ready for forcast modelling